In [5]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



#from secret import *

### Firefox Driver

In [31]:
wd = webdriver.Firefox()
#driver.implicitly_wait(10)
wd.get('http://www.afl.com.au/stats')

In [32]:
def data_download (_year,_round):
    movable_part = "CD_S{year}014&roundId=CD_R{year}014{round}".format(year=_year, round= _round)
    #print(movable_part)
    script = '' + \
    'window.a = jQuery.ajax({'+\
    '        url:"http://www.afl.com.au/api/cfs/afl/statsCentre/teams?competitionId={}",'.format(movable_part) +\
    '        headers: {'+\
    '                    "X-media-mis-token":"b0e22835816435c8436393d14e62ecf7"'+\
    '                 }'+\
    '        });' 

    wd.execute_script(script)
    print (_year, _round)
    import time
    import json
    time.sleep(1)
    wd.execute_script("console.log('the dude', window.a.responseText);")
    result = wd.execute_script("return window.a.responseText")
    
    try:
        data = json.loads(result)
        filename= _year  + '_' + _round + '.txt'
        with open('afl_raw/' + filename, 'w') as f:
            f.write(result)
            print ('exported ' + filename)
        
    except Exception as e:
        filename= _year  + '_' + _round + '.txt'
        with open('afl_raw/' + filename, 'w') as f:
            f.write('error occured on writing')
            print ('export error on ' + filename)
        

            

    

jQuery.ajax({url:"http://www.afl.com.au/api/cfs/afl/statsCentre/teams?competitionId=CD_S2017014&roundId=CD_R201701406", headers: {"X-media-mis-token":"b0e22835816435c8436393d14e62ecf7"}});

jQuery.ajax({url:" http://www.afl.com.au/api/cfs/afl/statsCentre/teams?competitionId=CD_S2003014&roundId=CD_R200301401", headers: {"X-media-mis-token":"b0e22835816435c8436393d14e62ecf7"}});

In [38]:
year_list = ['2017','2016','2015','2014','2013','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003','2002', '2001']
year_list_2 = ['2017','2016']
round_list = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27']

In [33]:
for i in year_list:
    current_year = i
    for i in round_list:
        current_round = i
        data_download(current_year,current_round)

2017 01
exported 2017_01.txt
2017 02
exported 2017_02.txt
2017 03
exported 2017_03.txt
2017 04
exported 2017_04.txt
2017 05
exported 2017_05.txt
2017 06
exported 2017_06.txt
2017 07
exported 2017_07.txt
2017 08
export error on 2017_08.txt
2017 09
export error on 2017_09.txt
2017 10
export error on 2017_10.txt
2017 11
export error on 2017_11.txt
2017 12
export error on 2017_12.txt
2017 13
export error on 2017_13.txt
2017 14
export error on 2017_14.txt
2017 15
export error on 2017_15.txt
2017 16
export error on 2017_16.txt
2017 17
export error on 2017_17.txt
2017 18
export error on 2017_18.txt
2017 19
export error on 2017_19.txt
2017 20
export error on 2017_20.txt
2017 21
export error on 2017_21.txt
2017 22
export error on 2017_22.txt
2017 23
export error on 2017_23.txt
2017 24
export error on 2017_24.txt
2017 25
export error on 2017_25.txt
2017 26
export error on 2017_26.txt
2017 27
export error on 2017_27.txt
2016 01
exported 2016_01.txt
2016 02
exported 2016_02.txt
2016 03
exported 20

### TESTING INDIVIDUAL DOWNLOADS

In [149]:
current_year = '2005'
current_round = '05'

In [150]:
data_download(current_year,current_round)

2005 05
exported 2005_05.txt


In [60]:
match_dict= {0:'01',1:'01',2:'02',3:'02', 4:'03',5:'03', 6:'04',7:'04',8:'05',9:'05', 10:'06',11:'06',12:'07',13:'07', 14:'08',15:'08', 16:'09',17:'09'}
print (match_dict[3])

02


### Function: Sort Data for each Round (file)

In [69]:
def structure_data(d, i, _year, _round):
    team_stats= d['team']
    match_stats = d['stats']['totals']
    stats = {**team_stats, **match_stats}
    stats['year'] = _year
    stats['round'] = _round
    # Finding match_id
    #string formatting mini lanuage
    match_dict= {0:'01',1:'01',2:'02',3:'02', 4:'03',5:'03', 6:'04',7:'04',8:'05',9:'05', 10:'06',11:'06',12:'07',13:'07', 14:'08',15:'08', 16:'09',17:'09'}
    #match_number = "{0:0=2d}".format(int(math.ceil(i/2)*2))
    match_number = (match_dict[i])
    stats['match_id'] = 'CD_M' + _year + '014'+  _round + str(match_number)
    #CD_M20170140401
    stats['home'] = i % 2 == 0
    stats = {**stats, **stats['clearances']}
    stats.pop('clearances', None)
    return stats

In [70]:
def readfile(_year, _round):
    filename= _year  + '_' + _round + '.txt'
    path = 'afl_raw/' + filename
    with open (path) as json_data: 
        data=json.load(json_data)
        #print (data['lists'])
        this_round = []
        for i, d in enumerate(data['lists']):
            stats = structure_data(d, i,_year, _round)
            this_round.append(stats)
        return this_round
        
            

In [71]:
mega_list = []
for current_year in year_list:
    for current_round in round_list:
        try:
            data = readfile(current_year, current_round)
            mega_list.extend(readfile(current_year, current_round))
            print (current_year)
        except Exception as e:
            print (e)


2017
2017
2017
2017
2017
Expecting value: line 1 column 1 (char 0)
2017
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
201

In [72]:
df = pd.DataFrame(mega_list)
print (df.shape)
df.head(300)

(6236, 48)


,behinds,bounces,centreClearances,clangers,contestedMarks,contestedPossessions,disposalEfficiency,disposals,dreamTeamPoints,freesAgainst,...,tacklesInside50,teamAbbr,teamId,teamName,teamNickname,totalClearances,totalPossessions,turnovers,uncontestedPossessions,year
0,5.0,8.0,19.0,49.0,14.0,138.0,73.6,348.0,1604.0,12.0,...,12.0,CARL,CD_T30,Carlton,Blues,38.0,341.0,81.0,203.0,2017
1,12.0,3.0,16.0,55.0,11.0,149.0,79.0,352.0,1551.0,25.0,...,23.0,RICH,CD_T120,Richmond,Tigers,40.0,354.0,68.0,205.0,2017
2,14.0,2.0,16.0,45.0,12.0,161.0,74.5,431.0,1835.0,16.0,...,12.0,COLL,CD_T40,Collingwood,Magpies,42.0,437.0,70.0,276.0,2017
3,10.0,3.0,10.0,44.0,12.0,135.0,77.0,392.0,1774.0,17.0,...,11.0,WB,CD_T140,Western Bulldogs,Bulldogs,26.0,394.0,63.0,259.0,2017
4,12.0,6.0,14.0,51.0,9.0,139.0,69.7,337.0,1341.0,15.0,...,11.0,STK,CD_T130,St Kilda,Saints,31.0,338.0,62.0,199.0,2017
5,12.0,2.0,19.0,60.0,9.0,147.0,77.5,454.0,1905.0,20.0,...,8.0,MELB,CD_T90,Melbourne,Demons,44.0,453.0,67.0,306.0,2017
6,10.0,2.0,10.0,61.0,10.0,139.0,71.8,358.0,1547.0,27.0,...,7.0,SYD,CD_T160,Sydney Swans,Swans,39.0,357.0,69.0,218.0,2017
7,8.0,3.0,18.0,69.0,12.0,161.0,69.3,339.0,1476.0,29.0,...,8.0,PORT,CD_T110,Port Adelaide,Power,49.0,339.0,72.0,178.0,2017
8,12.0,18.0,21.0,51.0,11.0,155.0,71.7,367.0,1577.0,19.0,...,15.0,GCFC,CD_T1000,Gold Coast Suns,Suns,41.0,376.0,72.0,221.0,2017
9,8.0,4.0,11.0,54.0,8.0,159.0,65.5,342.0,1447.0,20.0,...,15.0,BL,CD_T20,Brisbane Lions,Lions,33.0,329.0,70.0,170.0,2017


### EXPORTING TO CSV

In [65]:
df.to_csv("test3.csv")

### MERGING TWO DICTIONARIES AND ADDING TO DATAFRAME

a = {"b":1, "c": 2}

d = {"b":3, "c": 2}

x = [a,d]

pd.DataFrame(x)